In [116]:
from pinecone import Pinecone, PodSpec
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

connect = Pinecone(api_key = "26cf4e76-f2ca-4895-82af-525391c2932b")

In [117]:
from transformers import CLIPProcessor, CLIPModel

processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")

In [118]:
from PIL import Image
import torch

def get_image_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt", padding=True)
    # The following line is unnecessary for CLIP models and can be removed.
    inputs["input_ids"] = torch.ones(1, 1, dtype=torch.long)
    outputs = model(**inputs)
    # Use the correct attribute to access the image embeddings
    image_embedding = outputs.image_embeds.cpu().detach().numpy()

    return image_embedding

In [119]:
# creates pinecone index based on the dimensions of the model
def create_index_with_dim(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt", padding=True)
    inputs["input_ids"] = torch.ones(1, 1, dtype=torch.long)
    outputs = model(**inputs)
    image_embedding = outputs.image_embeds.cpu().detach().numpy()
    image_embed_shape = image_embedding.shape[1]

    # creates pinecone index based on the dimensions of the model
    connect.create_index(
      name = 'image-embeddings',
      dimension = image_embed_shape,
      metric = 'cosine',
      spec = PodSpec(
          environment = 'gcp-starter'
      )
    )

In [120]:
index = connect.Index('image-embeddings')

In [121]:
def upload_embeddings(image_paths, metadata):
    embeddings = [get_image_embedding(path) for path in image_paths]
    records = []
    for i, embedding in enumerate(embeddings):
        record = {"id": f"image_{i}", "values": embedding.tolist()[0], "metadata": metadata[i]}
        records.append(record)

    # upload records to Pinecone
    index.upsert(records)

In [122]:
import os

def process_folder(folder_path):
    image_paths = []
    metadata = []
    # counter for unique keys in metadata
    counter = 1

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # to be ablee too add any image type
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                image_path = os.path.join(root, file)
                image_paths.append(image_path)
                metadata.append({f"name{counter}": file})
                counter += 1
    # metadata added to see name of image but search based on unique id
    return image_paths, metadata

In [123]:
folder_path = '/Users/maximizosimov/Desktop/DS4300/hw6_test/test'
image_paths, metadata = process_folder(folder_path)

In [124]:
# creating index
create_index_with_dim(image_paths[0])

Unused or unrecognized kwargs: padding.


In [125]:
# adding embeddings to index in Pinecone
upload_embeddings(image_paths, metadata)

Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.


In [ ]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 6e-05,
 'namespaces': {'': {'vector_count': 6}},
 'total_vector_count': 6}

In [ ]:
def search_similar_images(query_image_path):
    query_embedding = get_image_embedding(query_image_path)[0]
    query_embedding = query_embedding.tolist()

    # search for similar images in the index
    results = index.query(
        vector=query_embedding,
        top_k=10
    )

    # print ids of similar images
    return results

In [ ]:
search_similar_images("/Users/maximizosimov/Desktop/DS4300/hw6_test/test_sample/laptop.jpg")

Unused or unrecognized kwargs: padding.


{'matches': [{'id': 'image_3', 'score': 1.00214231, 'values': []},
             {'id': 'image_5', 'score': 0.77876997, 'values': []},
             {'id': 'image_0', 'score': 0.596398413, 'values': []},
             {'id': 'image_1', 'score': 0.570945799, 'values': []},
             {'id': 'image_2', 'score': 0.530535, 'values': []},
             {'id': 'image_4', 'score': 0.493359268, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

In [ ]:
# put threshold to be below 1.01 or smth similar to reduce computing of recommendation per image